In [ ]:
import pandas as pd
import nltk
nltk.usage(nltk.classify.ClassifierI)
import nltk
import pandas as pd
import re
import random
import matplotlib.pyplot as plt
import numpy as np
from nltk.corpus import stopwords
#transform
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
#classifiers
from nltk.classify import SklearnClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import BernoulliNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import SGDClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix

#utilities
from sklearn.pipeline import Pipeline
from sklearn import metrics
from sklearn.model_selection import GridSearchCV

#graphing
import seaborn as sns

In [ ]:
def assign_mood(row):
    if row['energy'] >= 0.8 and row['valence'] >= 0.04 and row['danceability'] >= 0.4:
        return 'energetic'
    elif row['energy'] < 0.2240 and row['valence'] < 0.393 and row['danceability'] < 0.507:
        return 'sad'
    elif row['energy'] >= 0.7 and row['valence'] >= 0.7 and row['danceability'] >= 0.6:
        return 'happy'
    elif row['energy'] <= 0.320 and row['valence'] >= 0.3 and row['danceability'] >= 0.572:
        return 'romantic'
    elif row['energy'] >= 0.000282 and row['valence'] >= 0.01 and row['danceability'] >= 0.0499:
        return 'inspiring'
    elif row['energy'] > 0.2 and row['valence'] >= 0.6 and row['danceability'] >= 0.5:
        return 'hopeful'
    elif row['energy'] > 0.324 and row['valence'] >= 0.03 and row['danceability'] >= 0.2:
        return 'calm'
    elif row['energy'] < 0.324 and row['valence'] <= 0.239 and row['danceability'] <= 0.658:
        return 'reflective'
    elif row['energy'] >= 0.858 and row['valence'] < 0.682 and row['danceability'] < 0.737:
        return 'upbeat'

    # Add more conditions for other mood types

In [ ]:
df_m = pd.read_csv('df_comb_with_mood.csv')

In [ ]:
df_m.head(1)

In [ ]:
df_m.isnull().sum()

In [ ]:
import pandas as pd
import seaborn as sns

# Convert the 'mood' column to categorical data type
df_m['Mood'] = pd.Categorical(df_m['Mood'])

# Plot the count of each mood category
sns.countplot(data=df_m, x='Mood')

# Display the plot
plt.show()

In [ ]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score, cross_val_predict
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
import numpy as np

In [ ]:
features_for_mood = ['energy', 'danceability', 'valence']


hyper_opt = False

#split into trainval and test
trainx, testx, trainy, testy = train_test_split(df_m[features_for_mood], df_m['Mood'], test_size = 0.33,
                                                random_state = 42, stratify=df_m['Mood'])

scaler = StandardScaler()
train_scaled = scaler.fit_transform(trainx)

nn = MLPClassifier(max_iter = 15000, alpha=1.0, hidden_layer_sizes=8)
scores = cross_val_score(nn, train_scaled, trainy, cv=5)
print ("cv score: " + str(scores.mean()))

if hyper_opt:
    params = {"alpha": np.logspace(-4, 2, 7), 'hidden_layer_sizes': [1, 2, 5, 10, 20, 50, 100]}
    clf = GridSearchCV(nn, params)
    clf.fit(train_scaled, trainy)
    print("hyperparam optimized score : " + str(clf.best_score_))
    import pdb
    pdb.set_trace()

In [ ]:
from sklearn.model_selection import cross_validate

In [ ]:
results = cross_validate(nn, train_scaled, trainy, return_train_score=True)

In [ ]:
results

In [ ]:
nn = MLPClassifier(hidden_layer_sizes=8, max_iter=15000, alpha=1.0)

In [ ]:
nn.fit(train_scaled, trainy)
test_preds = nn.predict(scaler.transform(testx))
accuracy_score(test_preds, testy)

In [ ]:
import seaborn as sn
from sklearn.metrics import confusion_matrix

In [ ]:
conf_matrix = confusion_matrix(testy, test_preds, labels = df_m['Mood'].unique().tolist())

In [ ]:
labels = df_m['Mood'].unique().tolist()

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sn

plt.rcParams['figure.figsize'] = (7, 5)
# Define the labels

# Create the subplot and plot the confusion matrix
ax = plt.subplot()
sn.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues')

# Set the labels and title
ax.set_xlabel('Predicted labels')
ax.set_ylabel('True labels')
ax.set_title('Confusion Matrix')

# Set the tick labels
ax.xaxis.set_ticklabels(labels)
ax.yaxis.set_ticklabels(labels)

# Display the plot
plt.show()

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
plt.rcParams['figure.figsize'] = (5,5)

In [ ]:
from sklearn.model_selection import learning_curve
train_sizes, train_scores, val_scores = learning_curve(nn, train_scaled, trainy, cv=5)

In [ ]:
train_scores_mean = np.mean(train_scores, axis=1)
val_scores_mean = np.mean(val_scores, axis=1)
plt.plot(train_sizes, train_scores_mean, label='training score')
plt.plot(train_sizes, val_scores_mean, label='validation score')
plt.xlabel('training size')
plt.legend()



---

#RandomForest Classiefier

In [ ]:
#Importing Libraries Execute the following script to import the required libraries
import numpy as np
import re
import nltk
from sklearn.datasets import load_files
nltk.download('stopwords')
import pickle
from nltk.corpus import stopwords

In [ ]:
#Importing the Dataset
music_data =pd.read_csv('df_comb_with_mood.csv')
X, y = music_data['clean lyric'], music_data['Mood']

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer(max_features=1500, min_df=5, max_df=0.7, stop_words=stopwords.words('english'))
X = vectorizer.fit_transform(music_data['clean lyric']).toarray()

In [ ]:
from sklearn.feature_extraction.text import TfidfTransformer
tfidfconverter = TfidfTransformer()
X = tfidfconverter.fit_transform(X).toarray()

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [ ]:
from sklearn.ensemble import RandomForestClassifier

classifier = RandomForestClassifier(n_estimators=1000, random_state=0)
classifier.fit(X_train, y_train)

In [ ]:
y_pred = classifier.predict(X_test)

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

print(confusion_matrix(y_test,y_pred))
print(classification_report(y_test,y_pred))
print(accuracy_score(y_test, y_pred))



---



---



In [ ]:
import pandas as pd

#Importing the Dataset
music_data = pd.read_csv('df_comb_with_mood.csv')


In [ ]:
col_names = ["danceability", "valence", "energy"]

music_data = pd.read_csv('df_comb_with_mood.csv')

In [ ]:
# Mapping class label to respective integer
y = music_data['Mood'].map({'energetic': 0, 'happy': 1, 'inspiring': 2, 'romantic': 3, 'sad': 4})
target_names = ['energetic', 'happy', 'inspiring', 'romantic', 'sad']

In [ ]:
music_data.shape

In [ ]:
music_data['Mood'].value_counts()

In [ ]:
music_data['Mood'].value_counts(normalize=True)

In [ ]:
music_data['Mood'].value_counts().plot.bar();

In [ ]:
music_data.describe().T

In [ ]:
def clean_lyrics(lyric):
    # Your preprocessing code here (e.g., remove punctuations, convert to lowercase)
    lyric = re.sub(r'[^\w\s]', '', lyric)  # Remove punctuations
    return lyric.lower()  # Convert to lowercase
music_data['Lyric'] = music_data['Lyric'].apply(clean_lyrics)

In [ ]:
music_data['Lyric'] = music_data['Lyric'].apply(clean_lyrics)

In [ ]:
# Split the dataset into training and test sets
X_train, X_test, y_train, y_test = train_test_split(music_data['Lyric'], music_data['Mood'], test_size=0.2, random_state=42)

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
# Create a TF-IDF vectorizer and fit it to the training data
vectorizer = TfidfVectorizer()
X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)

In [ ]:
xtrain_samples = X_train.shape[0]
xtest_samples = X_test.shape[0]

print(f'There are {xtrain_samples} samples for training and {xtest_samples} samples for testing.')

In [ ]:
from sklearn.svm import SVC
svc = SVC(kernel='linear')

In [ ]:
svc.fit(X_train_tfidf, y_train)

In [ ]:
y_pred = svc.predict(X_test_tfidf)

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix

cm = confusion_matrix(y_test,y_pred)
sns.heatmap(cm, annot=True, fmt='d').set_title('Confusion matrix of linear SVM') # fmt='d' formats the numbers as digits, which means integers

print(classification_report(y_test,y_pred))



---



---

